# Mask R-CNN - Train on OpenImages Dataset

## Basic imports and constants

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from keras.callbacks import EarlyStopping

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

from openimages2019 import setup as st
from openimages2019 import utils as u

from skimage.draw import rectangle

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

DATA_DIR = os.path.join(ROOT_DIR, "../data")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(DATA_DIR, "mask_rcnn_coco.h5")

#Make GPUs visible
!export HIP_VISIBLE_DEVICES=1,2,3

#Set which GPU devices' memory should be accessible to running GPUs
os.environ["CUDA_VISIBLE_DEVICES"]="1,2,3"



import mlflow
#add mlflow stuff

MLFLOW_SERVER = 'occ01ap200.na.simplot.com'

os.environ['NO_PROXY'] = MLFLOW_SERVER
mlflow.tracking.set_tracking_uri('http://' + MLFLOW_SERVER + ':5005')
EXPERIMENT_NAME = 'kaggle_openimage_mask_rcnn_v1.0'
mlflow.set_experiment(EXPERIMENT_NAME)

# os.environ['AZURE_STORAGE_ACCESS_KEY'] = ''


def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## General Train functions

In [ ]:
class TrainConfig(Config):
    
    NAME = "kaggle"
    GPU_COUNT = 3
    IMAGES_PER_GPU = 3
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512
    

def log_params(pz):
    for k,v in pz.items():
        mlflow.log_param(k,v)

        

def train(model, inf_config, train_data, val_data, params):
    with mlflow.start_run():
        
#         es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
        
        model.train(train_data, val_data, 
                    learning_rate=params['learning_rate'], 
                    epochs=params['epochs'],
                    layers=params['layers']
#                     layers=params['layers'],
#                     custom_callbacks=[es]
                   )

        log_params(params)
        
        #Inference to get mAP
        inf_model = modellib.MaskRCNN(mode="inference", config=inf_config, model_dir=MODEL_DIR)
        model_path = inf_model.find_last()
        inf_model.load_weights(model_path, by_name=True)
        mAP = u.eval_mAP(inf_model, val_data, inf_config, params['mAP_sample_size'])
        
        mlflow.log_metric('mAP', mAP)
        mlflow.log_param('Model Path', model_path)
        
        # pickle.dump(clf,open(model_filepath,'wb'))
        # mlflow.log_artifact(model_filepath)
        
        return model_path, mAP

    
    
def train_on_class_subset(class_set, params, init_with="coco"):
    
    class KaggleConfig(TrainConfig):
#         STEPS_PER_EPOCH = params['STEPS_PER_EPOCH'] if 'STEPS_PER_EPOCH' in params else 500
        NUM_CLASSES = len(class_set) + 1 # + 1 for background class

    class InferenceConfig(KaggleConfig):
        GPU_COUNT = 1

    train_config = KaggleConfig()
    inf_config = InferenceConfig()    

    model = modellib.MaskRCNN(mode="training", config=train_config, model_dir=MODEL_DIR)

    if init_with == "imagenet":
        model.load_weights(model.get_imagenet_weights(), by_name=True)
    elif init_with == "coco":
        model.load_weights(COCO_MODEL_PATH, by_name=True,
                           exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                    "mrcnn_bbox", "mrcnn_mask"])
    elif init_with == "last":
        model.load_weights(model.find_last(), by_name=True)
    
    anns = st.load_annotations_by_image(class_set)
    train_data = st.load_dataset(anns, DATA_DIR, class_set, is_train=True)
    val_data = st.load_dataset(anns, DATA_DIR, class_set, is_train=False)

    return train(model, inf_config, train_data, val_data, params)


## Partition the classes according to frequency

In [ ]:
class_sets = st.partition_classes()

### Determine number of images containing any object of a class set

In [ ]:
anns = st.load_annotations_by_image()

val_anns = anns[anns['RelativePath'].str.contains('validation',regex=False)]
train_anns = anns[anns['RelativePath'].str.contains('train',regex=False)]

def count_em(anns):
    rval = []
    
    for cs in class_sets:
        z = anns[anns['LabelName'].isin(cs['LabelName'].values)]['ImageID'].nunique()
        rval.append(z)
        
    return rval

num_class_set_images_train = count_em(train_anns)
num_class_set_images_val = count_em(val_anns)

display(num_class_set_images_train)
display(num_class_set_images_val)

## Train a model on each partition

### Train all models

In [ ]:
params = {
        'learning_rate' : 0.001,
        'epochs' : 100,
        'layers' : 'heads',
        'mAP_sample_size' : 250,
        'GPU_COUNT' : 3,
        'IMAGES_PER_GPU' : 4,
        'IMAGE_MIN_DIM' : 512,
        'IMAGE_MAX_DIM' : 512
         }

#TODO log the entire Config object with mlflow, not just params
for i,class_set in enumerate(class_sets):
    params['class_set_index'] = i
    train_on_class_subset(class_set, params)


### Train a single model

In [ ]:
#Note that changes to ALL_CAPS properties in params do not affect the model.  You will need to change the corresponding
# values above (e.g in TrainConfig) to have any effect.

params = {
        'learning_rate' : 0.001,
        'epochs' : 300,
        'layers' : 'all',
        'mAP_sample_size' : 250,
        'GPU_COUNT' : 3,
        'IMAGES_PER_GPU' : 4,
        'IMAGE_MIN_DIM' : 512,
        'IMAGE_MAX_DIM' : 512
         }

set_num = 2

params['class_set_index'] = set_num
train_on_class_subset(class_sets[set_num], params)

### Train a subset

In [ ]:
params = {
        'learning_rate' : 0.001,
        'epochs' : 200,
        'layers' : 'heads',
        'mAP_sample_size' : 250,
        'GPU_COUNT' : 3,
        'IMAGES_PER_GPU' : 3,
        'IMAGE_MIN_DIM' : 512,
        'IMAGE_MAX_DIM' : 512
         }


for i in [1,3,4,5]:
    params['class_set_index'] = i
    train_on_class_subset(class_sets[i], params)